In [21]:
# data analysis stack
import numpy as np
import pandas as pd

# data visualization stack
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

# miscellaneous
import warnings
warnings.filterwarnings("ignore")
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

# deep learning stack
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Model, Sequential

from tensorflow.keras.applications.resnet50 import (
    ResNet50,
    preprocess_input,
    decode_predictions
)

In [22]:
CLASSES = os.listdir('./cancer/Cancer/')
CLASSES

['ACK', 'BCC', 'MEL', 'NEV', 'SCC', 'SEK']

In [23]:
data_gen = image.ImageDataGenerator(
    preprocessing_function=preprocess_input
)
train_data_gen = data_gen.flow_from_directory(
        directory='./cancer/Cancer/',
        class_mode="categorical",
        classes=CLASSES,
        batch_size=900,
        target_size=(224,224)
)

xtrain, ytrain = next(train_data_gen)
xtrain.shape, ytrain.shape

Found 900 images belonging to 6 classes.


((900, 224, 224, 3), (900, 6))

In [24]:
# Define the CNN model
BASE_MODEL = ResNet50(
    weights='imagenet',
    include_top=False,
    pooling='avg',
    input_shape=(224, 224, 3)
)
BASE_MODEL.trainable = False

In [25]:
def HyperModel(hyperparameters):
    '''
    creates a hypermodel by stacking dense layers on top of base model.
    Three hyperparameters to be tuned:
    1. number of neurons in the first dense layer,
    2. number of neurons in the second dense layer,
    3. number of neurons in the third dense layer,
    4. initial learning rate of the optimizer
    Args:
    hyperparameters - Keras tuner object
    '''
    # initialize the Sequential API to stack the layers
    model = keras.Sequential()
    
    # convolutional base 
    model.add(BASE_MODEL)
    
    # number of neurons in the first dense layer
    hp_units1 = hyperparameters.Choice('units1', values=[2048,512,256])
    # first dense layer
    model.add(keras.layers.Dense(units=hp_units1, activation='relu'))
    
    # number of neurons in the second dense layer
    hp_units2 = hyperparameters.Choice('units2', values=[512,256,128])
    # second dense layer
    model.add(keras.layers.Dense(units=hp_units2, activation='relu'))
    
    # number of neurons in the third dense layer
    hp_units3 = hyperparameters.Choice('units3', values=[[256,128,64,32,16]])
    # third dense layer
    model.add(keras.layers.Dense(units=hp_units3, activation='relu'))
    
    # dropout 
    model.add(keras.layers.Dropout(rate=0.5))
    
    # output layer with softmax activation function
    model.add(keras.layers.Dense(len(CLASSES), activation='softmax'))
    
    # learning rate for the optimizer
    hp_learning_rate = hyperparameters.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    
    # compile model
    model.compile(
        optimizer=keras.optimizers.Adam(
            learning_rate=hp_learning_rate
        ),
        loss=keras.losses.categorical_crossentropy,
        metrics=[keras.metrics.categorical_accuracy]
    )
    
    return model


In [26]:
import keras_tuner as kt
# instantiate hyperband
tuner = kt.GridSearch(
    hypermodel=HyperModel,
    objective='val_categorical_accuracy')

INFO:tensorflow:Reloading Tuner from .\untitled_project\tuner0.json


In [9]:
# hypertuning settings summary
tuner.search_space_summary() 

Search space summary
Default search space size: 4
units1 (Choice)
{'default': 2048, 'conditions': [], 'values': [2048, 512, 256], 'ordered': True}
units2 (Choice)
{'default': 512, 'conditions': [], 'values': [512, 256, 128], 'ordered': True}
units3 (Choice)
{'default': 256, 'conditions': [], 'values': [256, 128, 64, 32, 16], 'ordered': True}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [10]:
# early stopping
stop_early = keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    patience=10
)

In [11]:
tuner.search(
    xtrain, 
    ytrain,
    epochs=25,
    validation_split=0.2,
    callbacks=[stop_early]
)


Search: Running Trial #32

Value             |Best Value So Far |Hyperparameter
2048              |2048              |units1
128               |512               |units2
256               |128               |units3
0.001             |0.001             |learning_rate



Traceback (most recent call last):
  File "c:\Users\berka\anaconda3\envs\final_prj\lib\site-packages\keras_tuner\engine\base_tuner.py", line 270, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "c:\Users\berka\anaconda3\envs\final_prj\lib\site-packages\keras_tuner\engine\base_tuner.py", line 235, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "c:\Users\berka\anaconda3\envs\final_prj\lib\site-packages\keras_tuner\engine\tuner.py", line 287, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "c:\Users\berka\anaconda3\envs\final_prj\lib\site-packages\keras_tuner\engine\tuner.py", line 213, in _build_and_fit_model
    model = self._try_build(hp)
  File "c:\Users\berka\anaconda3\envs\final_prj\lib\site-packages\keras_tuner\engine\tuner.py", line 155, in _try_build
    model = self._build_hypermodel(hp)
  File "c:\Users\berka\anaconda3\envs\final_prj\lib\

RuntimeError: Number of consecutive failures excceeded the limit of 3.
Traceback (most recent call last):
  File "c:\Users\berka\anaconda3\envs\final_prj\lib\site-packages\keras_tuner\engine\base_tuner.py", line 270, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "c:\Users\berka\anaconda3\envs\final_prj\lib\site-packages\keras_tuner\engine\base_tuner.py", line 235, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "c:\Users\berka\anaconda3\envs\final_prj\lib\site-packages\keras_tuner\engine\tuner.py", line 287, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "c:\Users\berka\anaconda3\envs\final_prj\lib\site-packages\keras_tuner\engine\tuner.py", line 213, in _build_and_fit_model
    model = self._try_build(hp)
  File "c:\Users\berka\anaconda3\envs\final_prj\lib\site-packages\keras_tuner\engine\tuner.py", line 155, in _try_build
    model = self._bu

In [13]:
best_hp = tuner.get_best_hyperparameters(1)[0]

In [17]:
best_hp.get('units1')

2048

In [18]:
best_hp.get('units2')

512

In [19]:
best_hp.get('units3')

128

In [20]:
best_hp.get('learning_rate')

0.001

In [32]:
from tensorflow.keras.models import load_model
model = load_model("model_moons.h5")

In [33]:
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 53)]         0           []                               
                                                                                                  
 resnet50_input (InputLayer)    [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 densenet121_input (InputLayer)  [(None, 224, 224, 3  0          []                               
                                )]                                                                
                                                                                              

In [36]:
# losses in the training history
losses = pd.DataFrame(model.history.history)
# training loss
loss = losses['loss'].values 
# validation loss
val_loss = losses['val_loss'].values

# epoch number
epoch = losses.index.values + np.ones_like(losses.index.values)

AttributeError: 'NoneType' object has no attribute 'history'

In [ ]:
# plot losses against epoch number in log-scale
plt.figure(figsize=(8,4),dpi=100)
plt.semilogx(epoch, loss, linewidth=1.5, label='loss')
plt.semilogx(epoch, val_loss, linewidth=1.5, label='val_loss')

plt.xlabel('epoch',fontsize=10)
plt.ylabel('loss value',fontsize=10)
plt.legend(loc=1,fontsize=10);